In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
cfb = pd.read_csv('CFBData.csv')

In [4]:
cfb['id'] = cfb['name'] + '_' + cfb['year'].astype(str)

def create_id(x):
    ids = x.lower().split(' ')
    return '_'.join(ids)

cfb['id'] = cfb['id'].apply(create_id)

In [7]:
#drop any stats relating to wins, games, and any categorical
cfb_stats = cfb[['first_downs',
       'opponents_first_downs', 'first_downs_from_penalties',
       'opponents_first_downs_from_penalties', 'fumbles_lost',
       'opponents_fumbles_lost', 'interceptions',
       'opponents_interceptions', 'pass_attempts',
       'opponents_pass_attempts', 'pass_completion_percentage',
       'opponents_pass_completion_percentage', 'pass_completions',
       'opponents_pass_completions', 'pass_first_downs',
       'opponents_pass_first_downs', 'pass_touchdowns',
       'opponents_pass_touchdowns', 'pass_yards', 'opponents_pass_yards',
       'penalties', 'opponents_penalties', 'plays', 'opponents_plays',
       'points_against_per_game', 'points_per_game', 'rush_attempts',
       'opponents_rush_attempts', 'rush_first_downs',
       'opponents_rush_first_downs', 'rush_touchdowns',
       'opponents_rush_touchdowns', 'rush_yards', 'opponents_rush_yards',
       'rush_yards_per_attempt', 'opponents_rush_yards_per_attempt',
       'simple_rating_system', 'strength_of_schedule', 'turnovers',
       'opponents_turnovers', 'yards',
       'opponents_yards', 'yards_from_penalties',
       'opponents_yards_from_penalties', 'yards_per_play',
       'opponents_yards_per_play']].dropna()

### BASELINE

In [6]:
from sklearn.linear_model import LinearRegression
X = cfb_stats.drop('simple_rating_system', axis = 1)
y = cfb_stats['simple_rating_system']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)
baseline = LinearRegression()
baseline.fit(X_train, y_train)
y_pred = baseline.predict(X_test)

In [8]:
baseline = LinearRegression()
baseline.fit(X_train, y_train)
y_pred = baseline.predict(X_test)

In [9]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
print('mse', mean_squared_error(y_test, y_pred))
print('mae', mean_absolute_error(y_test, y_pred))

mse 2.5204027090418504
mae 1.2826733675648974


### Model

In [11]:
cfb_stats['first_downs_diff'] = cfb_stats['first_downs'] - cfb_stats['opponents_first_downs']
cfb_stats['first_downs_from_penalties_diff'] = cfb_stats['first_downs_from_penalties'] - cfb_stats['opponents_first_downs_from_penalties']
cfb_stats['fumbles_lost_diff'] = cfb_stats['fumbles_lost'] - cfb_stats['opponents_fumbles_lost']
cfb_stats['interceptions_diff'] = cfb_stats['interceptions'] - cfb_stats['opponents_interceptions']
cfb_stats['pass_attempts_diff'] = cfb_stats['pass_attempts'] - cfb_stats['opponents_pass_attempts']
cfb_stats['pass_completion_percentage_diff'] = cfb_stats['pass_completion_percentage'] - cfb_stats['opponents_pass_completion_percentage']
cfb_stats['pass_completion_diff'] = cfb_stats['pass_completions'] - cfb_stats['opponents_pass_completions']
cfb_stats['pass_first_downs_diff'] = cfb_stats['pass_first_downs'] - cfb_stats['opponents_pass_first_downs']
cfb_stats['pass_touchdowns_diff'] = cfb_stats['pass_touchdowns'] - cfb_stats['opponents_pass_touchdowns']
cfb_stats['pass_yards_diff'] = cfb_stats['pass_yards'] - cfb_stats['opponents_pass_yards']
cfb_stats['penalties_diff'] = cfb_stats['penalties'] - cfb_stats['opponents_penalties']
cfb_stats['plays_diff'] = cfb_stats['plays'] - cfb_stats['opponents_plays']
cfb_stats['score_differential'] = cfb_stats['points_per_game'] - cfb_stats['points_against_per_game']
cfb_stats['rush_attempts_diff'] = cfb_stats['rush_attempts'] - cfb_stats['opponents_rush_attempts']
cfb_stats['rush_first_downs_diff'] = cfb_stats['rush_first_downs'] - cfb_stats['opponents_rush_first_downs']
cfb_stats['rush_touchdowns_diff'] = cfb_stats['rush_touchdowns'] - cfb_stats['opponents_rush_touchdowns']
cfb_stats['rush_yards_diff'] = cfb_stats['rush_yards'] - cfb_stats['opponents_rush_yards']
cfb_stats['rush_yards_per_attempt_diff'] = cfb_stats['rush_yards_per_attempt'] - cfb_stats['opponents_rush_yards_per_attempt']
cfb_stats['turnovers_diff'] = cfb_stats['turnovers'] - cfb_stats['opponents_turnovers']
cfb_stats['yards_diff'] = cfb_stats['yards'] - cfb_stats['opponents_yards']
cfb_stats['yards_from_penalties_diff'] = cfb_stats['yards_from_penalties'] - cfb_stats['opponents_yards_from_penalties']
cfb_stats['yards_per_play_diff'] = cfb_stats['yards_per_play'] - cfb_stats['opponents_yards_per_play']

In [12]:
cfb_diff = cfb_stats.drop(['first_downs', 'opponents_first_downs', 'first_downs_from_penalties',
       'opponents_first_downs_from_penalties', 'fumbles_lost',
       'opponents_fumbles_lost', 'interceptions', 'opponents_interceptions',
       'pass_attempts', 'opponents_pass_attempts',
       'pass_completion_percentage', 'opponents_pass_completion_percentage',
       'pass_completions', 'opponents_pass_completions', 'pass_first_downs',
       'opponents_pass_first_downs', 'pass_touchdowns',
       'opponents_pass_touchdowns', 'pass_yards', 'opponents_pass_yards',
       'penalties', 'opponents_penalties', 'plays', 'opponents_plays',
       'points_against_per_game', 'points_per_game', 'rush_attempts',
       'opponents_rush_attempts', 'rush_first_downs',
       'opponents_rush_first_downs', 'rush_touchdowns',
       'opponents_rush_touchdowns', 'rush_yards', 'opponents_rush_yards',
       'rush_yards_per_attempt', 'opponents_rush_yards_per_attempt',
       'turnovers',
       'opponents_turnovers', 'yards', 'opponents_yards',
       'yards_from_penalties', 'opponents_yards_from_penalties',
       'yards_per_play', 'opponents_yards_per_play'], axis = 1)

In [13]:
cfb_diff['id'] = cfb['id']
cfb_diff['team'] = cfb['name']
cfb_diff['year'] = cfb['year']

In [14]:
prior_srs = cfb_diff.set_index(['team', 'year']).sort_values(['team', 'year']).shift(1)['simple_rating_system'].values

In [15]:
cfb_diff = cfb_diff.set_index(['team', 'year']).sort_values(['team', 'year'])
cfb_diff['prior_srs'] = prior_srs
cfb_diff = cfb_diff.reset_index()
cfb_diff = cfb_diff.drop(['team', 'year', 'id'], axis = 1).dropna()

In [16]:
X = cfb_diff.drop(['simple_rating_system', 'first_downs_from_penalties_diff', 'penalties_diff'], axis = 1)
y = cfb_diff['simple_rating_system']

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)
model = LinearRegression()
scaler = StandardScaler()
standardized_X_train = scaler.fit_transform(X_train)
standardized_X_test = scaler.transform(X_test)
model.fit(standardized_X_train, y_train)
y_pred = model.predict(standardized_X_test)

from sklearn.metrics import mean_squared_error
print('mse', mean_squared_error(y_test, y_pred))
print('mae', mean_absolute_error(y_test, y_pred))

mse 2.2182330692405707
mae 1.1982693406886153


In [18]:
from sklearn import linear_model
scaler = StandardScaler()
standardized_X_train = scaler.fit_transform(X_train)
standardized_X_test = scaler.transform(X_test)
model = linear_model.Lasso(alpha=0.1)
model.fit(standardized_X_train, y_train)
y_pred = model.predict(standardized_X_test)

from sklearn.metrics import mean_squared_error
print('mse', mean_squared_error(y_test, y_pred))
print('mae', mean_absolute_error(y_test, y_pred))

mse 2.359189074692517
mae 1.2454909568839538


In [19]:
from sklearn.model_selection import GridSearchCV
estimator = linear_model.Ridge(alpha=1)
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=5)
grid.fit(standardized_X_train, y_train)

print(grid.best_score_ , grid.best_params_)

0.9759282834580087 {'alpha': 1}


In [20]:
from sklearn import linear_model
scaler = StandardScaler()
standardized_X_train = scaler.fit_transform(X_train)
standardized_X_test = scaler.transform(X_test)
model = linear_model.Ridge(alpha = 1)
model.fit(standardized_X_train, y_train)
y_pred = model.predict(standardized_X_test)

from sklearn.metrics import mean_squared_error
print('mse', mean_squared_error(y_test, y_pred))
print('mae', mean_absolute_error(y_test, y_pred))

mse 2.2101614862157737
mae 1.1941271858172362


In [21]:
from sklearn.svm import SVR
estimator = SVR()
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1]}
grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=5)
grid.fit(standardized_X_train, y_train)

print(grid.best_score_ , grid.best_params_)

0.975396294975136 {'C': 100, 'gamma': 0.001}


In [26]:
model = SVR(C=100, gamma=.001)
scaler = StandardScaler()
standardized_X_train = scaler.fit_transform(X_train)
standardized_X_test = scaler.transform(X_test)
model.fit(standardized_X_train, y_train)
y_pred = model.predict(standardized_X_test)

from sklearn.metrics import mean_squared_error
print('mse', mean_squared_error(y_test, y_pred))
print('mae', mean_absolute_error(y_test, y_pred))

mse 2.2389553611147797
mae 1.1909591071337968


In [25]:
from sklearn.ensemble import RandomForestRegressor
estimator = RandomForestRegressor()
param_grid = { 
             "n_estimators": [10,20,30,100],
             "max_features": ["auto", "sqrt", "log2"],
             "min_samples_split" : [2,4,8],
             "bootstrap": [True, False],
             }
grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=5)
grid.fit(standardized_X_train, y_train)

print(grid.best_score_ , grid.best_params_)

0.9666330247392312 {'bootstrap': True, 'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 100}


In [27]:
model = RandomForestRegressor(bootstrap=True, max_features='auto', min_samples_split=2,n_estimators=100)
scaler = StandardScaler()
standardized_X_train = scaler.fit_transform(X_train)
standardized_X_test = scaler.transform(X_test)
model.fit(standardized_X_train, y_train)
y_pred = model.predict(standardized_X_test)

from sklearn.metrics import mean_squared_error
print('mse', mean_squared_error(y_test, y_pred))
print('mae', mean_absolute_error(y_test, y_pred))

mse 2.7614043388214284
mae 1.285415357142857


In [28]:
from xgboost import XGBRegressor as xgb
estimator = xgb()
param_grid = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}
grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=5)
grid.fit(standardized_X_train, y_train)

print(grid.best_score_ , grid.best_params_)

0.969689310057972 {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 140}


In [29]:
model = xgb(learning_rate=0.05, max_depth=4, n_estimators=140)
scaler = StandardScaler()
standardized_X_train = scaler.fit_transform(X_train)
standardized_X_test = scaler.transform(X_test)
model.fit(standardized_X_train, y_train)
y_pred = model.predict(standardized_X_test)

from sklearn.metrics import mean_squared_error
print('mse', mean_squared_error(y_test, y_pred))
print('mae', mean_absolute_error(y_test, y_pred))

mse 2.5725542772158527
mae 1.2734334258485056
